Database Connection

In [44]:
import sqlite3

conn = sqlite3.connect('cancer.db')

try:
    conn.cursor()
    print("DB Connected");
except Exception as ex:
    print("DB not Connected");

DB Connected


FOLLOWING TABLES ARE CREATED -
User Table : This table contains the details of the user, who have tweeted
Cancer Type Table : This table holds different categories of cancer
Treatment Table: This table contains types of treatment based on Cancer type
Location Table:This table describes the location of the user
Smoking Table: This table briefs whether user is a smoker or not

In [45]:
#USER TABLE
conn = sqlite3.connect('cancer.db')
print("Opened database successfully");
conn.execute('''DROP TABLE IF EXISTS USER;''')
conn.execute('''CREATE TABLE IF NOT EXISTS USER
         (
         User_Id INTEGER PRIMARY KEY AUTOINCREMENT,
         User_Name     TEXT   NOT NULL,
         Handle_Name   TEXT   NOT NULL,
         Description   TEXT   NOT NULL,
         Followers_Count     INTEGER,
         Created_Date   TEXT  NOT NULL,
         Image_Url      TEXT  NOT NULL
         );''')

print("User Table created successfully");

conn.close()

#CANCER_TYPE TABLE
conn = sqlite3.connect('cancer.db')
print("Opened database successfully");
conn.execute('''DROP TABLE IF EXISTS CANCER_TYPE;''')
conn.execute('''CREATE TABLE IF NOT EXISTS CANCER_TYPE
         (Hashtag_Id INTEGER NOT NULL,
         Cancer_Id INTEGER PRIMARY KEY AUTOINCREMENT,
         Cancer_Type      TEXT  NOT NULL,
         FOREIGN KEY(Hashtag_Id) REFERENCES HASHTAG(Hashtag_Id));''')

print("Cancer_Type Table created successfully");

conn.close()

#TREATMENT TABLE
conn = sqlite3.connect('cancer.db')
print("Opened database successfully");
conn.execute('''DROP TABLE IF EXISTS TREATMENT;''')
conn.execute('''CREATE TABLE IF NOT EXISTS TREATMENT
         (Cancer_Id INTEGER NOT NULL,
          Treatment_Id INTEGER PRIMARY KEY AUTOINCREMENT,
          Treatment_Type      TEXT  NOT NULL,
          FOREIGN KEY(Cancer_Id) REFERENCES CANCER_TYPE(Cancer_Id));''')

print("Treatment Table created successfully");

conn.close()

#LOCATION TABLE
conn = sqlite3.connect('cancer.db')
print("Opened database successfully");
conn.execute('''DROP TABLE IF EXISTS LOCATION;''')
conn.execute('''CREATE TABLE IF NOT EXISTS LOCATION
         (User_Id INTEGER NOT NULL ,
          Location_Id INTEGER PRIMARY KEY AUTOINCREMENT,
          Location    TEXT  NOT NULL,
          FOREIGN KEY(User_Id) REFERENCES USER(User_Id) );''')

print("Location Table created successfully");

conn.close()

#SMOKING TABLE
conn = sqlite3.connect('cancer.db')
print("Opened database successfully");
conn.execute('''DROP TABLE IF EXISTS SMOKING;''')
conn.execute('''CREATE TABLE IF NOT EXISTS SMOKING
         (Cancer_Id INTEGER NOT NULL,
          Smoking_Id INTEGER PRIMARY KEY AUTOINCREMENT,
          Smoker_Or_Not  INTEGER NOT NULL,
          FOREIGN KEY(Cancer_Id) REFERENCES CANCER_TYPE(Cancer_Id) );''')

print("Smoking Table created successfully");

conn.close()

Opened database successfully
User Table created successfully
Opened database successfully
Cancer_Type Table created successfully
Opened database successfully
Treatment Table created successfully
Opened database successfully
Location Table created successfully
Opened database successfully
Smoking Table created successfully


FOLLOWING TABLES ARE CREATED -
Tweets Table : This table contains the details of the tweet with the tweet handle name and the posted date
Hashtag Table : This table holds details on the hashtag and the tweet_id
Tweet mention Table: This table contains information about the tweet mention, source user and target user
Tweet URL Table: This table describes the URL of the tweet and the URL Id

In [46]:
#TWEETS TABLE
conn = sqlite3.connect('cancer.db')
print("Opened database successfully");
conn.execute('''DROP TABLE IF EXISTS TWEETS;''')
conn.execute('''CREATE TABLE IF NOT EXISTS TWEETS
         (Tweet_Id INTEGER PRIMARY KEY NOT NULL,
          Tweet_Handle TEXT NOT NULL,
          Tweet_Text TEXT NOT NULL,
          Posted_At TEXT,
          User_Id INTEGER NOT NULL,
          FOREIGN KEY(User_Id) REFERENCES USER(User_Id));''')

print("Tweets Table created successfully");

conn.close()

#HASHTAG TABLE
conn = sqlite3.connect('cancer.db')
print("Opened database successfully");
conn.execute('''DROP TABLE IF EXISTS HASHTAG;''')
conn.execute('''CREATE TABLE IF NOT EXISTS HASHTAG
         (Hashtag_Id INTEGER PRIMARY KEY NOT NULL,
          Hashtag_Text TEXT NOT NULL,
          Tweet_Id  INTEGER NOT NULL,
          FOREIGN KEY(Tweet_Id) REFERENCES TWEETS(Tweet_Id));''')

print("Hashtag Table created successfully");

conn.close()

#TWEET MENTION TABLE
conn = sqlite3.connect('cancer.db')
print("Opened database successfully");
conn.execute('''DROP TABLE IF EXISTS TWEET_MENTION;''')
conn.execute('''CREATE TABLE IF NOT EXISTS TWEET_MENTION
         (Mention_Id INTEGER PRIMARY KEY NOT NULL,
          Source_User TEXT NOT NULL,
          Target_User TEXT NOT NULL,
          Tweet_Id  INTEGER NOT NULL,
          FOREIGN KEY(Tweet_Id) REFERENCES TWEETS(Tweet_Id));''')

print("Tweet Mention Table created successfully");

conn.close()

#TWEET URL TABLE
conn = sqlite3.connect('cancer.db')
print("Opened database successfully");
conn.execute('''DROP TABLE IF EXISTS TWEET_URL;''')
conn.execute('''CREATE TABLE IF NOT EXISTS TWEET_URL
         (Url_Id INTEGER PRIMARY KEY NOT NULL,
          Tweet_Url TEXT NOT NULL,
          Tweet_Id  INTEGER NOT NULL,
          FOREIGN KEY(Tweet_Id) REFERENCES TWEETS(Tweet_Id));''')

print("Tweet URL Table created successfully");

conn.close()


Opened database successfully
Tweets Table created successfully
Opened database successfully
Hashtag Table created successfully
Opened database successfully
Tweet Mention Table created successfully
Opened database successfully
Tweet URL Table created successfully


Scrapping Twitter for #Cancer

In [73]:
from twitter_scraper_selenium import scrape_keyword
import json
from twitter_scraper_selenium import get_profile_details

cancer = json.loads(scrape_keyword(keyword="cancer", browser="chrome",
                                   tweets_count=1, output_format="json", until="2023-08-31", since="2022-08-30"))

for item in cancer:
    userName = cancer[item]['username'];

    #Fetching User profile Data
    filename = "twitter_api_data"
    data = json.loads(get_profile_details(twitter_username=userName))

    #Storing details which are to be inserted
    Handle_Name = data['name']
    Description = data['description']
    Followers_Count = data['followers_count']
    Created_Date = data['created_at']
    Image_Url = data['profile_image_url']
    Tweets_Handle = data['name']
    Tweets_Text = cancer[item]['content']
    Posted_At = cancer[item]['posted_time']
    Tweet_id = cancer[item]['tweet_id']
    location = 'NA' if data['profile_location'] == "" else data['profile_location']


    #Insert into USER table
    conn = sqlite3.connect('cancer.db')
    conn.execute(
        'INSERT INTO USER (User_Name,Handle_Name,Description,Followers_Count,Created_Date,Image_Url) VALUES (?,?,?,?,?,?)',
        (userName, Handle_Name, Description, Followers_Count, Created_Date, Image_Url))
    conn.commit()
    conn.close()

    #Fetch User_id from USER Table
    print(userName)
    User_id=0;
    conn = sqlite3.connect('cancer.db')
    cur = conn.cursor()
    cur.execute('SELECT User_Id FROM USER WHERE User_Name = ?', (userName,))
    rows = cur.fetchone()

    for row in rows:
        User_id=row

    conn.commit()
    conn.close()

    print("user id ->",User_id)


    #Insert into TWEETS table
    conn = sqlite3.connect('cancer.db')
    conn.execute(
        'INSERT INTO TWEETS (Tweet_Id,Tweet_Handle,Tweet_Text,Posted_At,User_Id) VALUES (?,?,?,?,?)',
        (Tweet_id, Handle_Name, Tweets_Text, Posted_At, User_id))
    conn.commit()
    conn.close()

    print(cancer[item]);

[WDM] - Current google-chrome version is 107.0.5304
[WDM] - Get LATEST driver version for 107.0.5304
[WDM] - Driver [/Users/vigy/.wdm/drivers/chromedriver/mac64/107.0.5304.62/chromedriver] found in cache


 
moxargon
user id -> 22
{'tweet_id': '1591290918476578817', 'username': 'moxargon', 'name': 'Remulak MoxArgon: Possibly A Parody', 'profile_picture': 'https://pbs.twimg.com/profile_images/2287382400/image_x96.jpg', 'replies': 0, 'retweets': 0, 'likes': 0, 'is_retweet': True, 'posted_time': '2022-11-12T04:44:54+00:00', 'content': 'I hope so, he’s been a cancer on American politics from the moment he declared his candidacy.', 'hashtags': [], 'mentions': [], 'images': [], 'videos': [], 'tweet_url': 'https://twitter.com/moxargon/status/1591290918476578817', 'link': ''}
